In [28]:
model_file = "./model/IdxDov2VecDeep_v5_7epoch.weight"
#testing_user = "./data/testing_users.csv"
testing_user = "./clean_data/test_data/test_before_021718_7000row_fold0.csv"
submission_file = "./submission/deep_learning_v5.csv"

## Predicting projects for user using deep learning

In [29]:
import pandas as pd
import torch

In [30]:
# Get all necessary data frame here
submission_users = pd.read_csv(testing_user)
userLog = pd.read_csv('./clean_data/userLog_beforeTest.csv')
item_feature = pd.read_csv('./clean_data/proj.csv')
user_feature = pd.read_csv('./clean_data/user_v2.csv')
docvec = pd.read_csv("./clean_data/doc2vec.csv")

In [31]:
user2IdxDf = pd.read_csv("./clean_data/user2Idx.csv")
proj2IdxDf  = pd.read_csv("./clean_data/proj2Idx.csv")

In [32]:
userEmbeddingNum = user2IdxDf.shape[0]
projEmbeddingNum = proj2IdxDf.shape[0]

In [33]:
## Deep Learning Here!
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

class Net(nn.Module):
    def __init__(self, doc2vecDim = 300):
        super(Net, self).__init__()        
        self.doc2VecDim = doc2vecDim        
        self.userEmbedding = nn.Embedding(userEmbeddingNum, 100)
        self.projEmbedding = nn.Embedding(projEmbeddingNum, 100)        
        self.dVecFc= nn.Linear(doc2vecDim, 300)                
        self.hidden = nn.Linear(500,300)
        self.hidden2 = nn.Linear(300,200)
        self.hidden3 = nn.Linear(200,200)
        self.fc2 = nn.Linear(200,1)

    def forward(self, x):

        userFeature, projFeature, dVecFeature = x[:,:1],x[:,1:2], x[:,-self.doc2VecDim:]
        try:
            userEmbedding = self.userEmbedding(userFeature.long())
            projEmbedding = self.projEmbedding(projFeature.long())
            dVecEmbedding = F.relu(self.dVecFc(dVecFeature))
        except e:
            print(e)
            print(userFeature.max(),projFeature.max())
        
        combined = torch.cat((userEmbedding.view(len(x),-1), projEmbedding.view(len(x),-1),dVecEmbedding.view(len(x),-1)),dim=1)
        x = self.hidden(combined)
        x = self.hidden2(x)
        x = self.hidden3(x)
        x = self.fc2(x)
        
        return x

model = Net()
model.load_state_dict(torch.load(model_file))

RuntimeError: While copying the parameter named userEmbedding.weight, whose dimensions in the model are torch.Size([482744, 100]) and whose dimensions in the checkpoint are torch.Size([482744, 150]).

In [7]:
# For each user, select multiple projects that pass some certain condition
# Current condition: same province and price range.
"""
Log_starting_price, province, (lat_lon distance)
"""
#submission_users_merged = submission_users.merge(user_feature,on='userCode')[['Log_starting_price',' province']]#['district_id']#.merge(item_feature,on='project_id')
from itertools import product

#Unleash hell here (Reload checking_df to be full size)
checking_df = pd.DataFrame(list(product(submission_users['userCode'], item_feature['project_id'])),columns=['userCode','project_id'])

In [8]:
%%time
# Merge the checking dataframe with user feature and item feature
checking_df_mergedd = checking_df.merge(user_feature,on='userCode').merge(item_feature, on='project_id',suffixes=("_user","_project")) # Takes 3 mins
checking_df_merged = checking_df_mergedd[['userCode','project_id','starting_price_range_user','starting_price_range_project','project_list','most_province_id','province_id_project']] # 50 s

CPU times: user 5min 31s, sys: 6min 1s, total: 11min 33s
Wall time: 6min 40s


### Filtering

In [9]:
#item_feature[item_feature['project_id']==8997]

In [10]:
%%time
# Filter out rows that do not statisfied the conditions


# samePriceRange = checking_df_merged['starting_price_range_user'] == checking_df_merged['starting_price_range_project']
# checking_df_merged_filtered = checking_df_merged[samePriceRange]

# sameProvince = checking_df_merged_filtered['province_id_project']==checking_df_merged_filtered['most_province_id']
# checking_df_merged_filtered = checking_df_merged_filtered[sameProvince]

checking_df_merged_filtered = checking_df_merged

print(checking_df_merged_filtered.shape)
# Filter out rows that with project that user has already seen
hasOldProject = checking_df_merged_filtered.apply(lambda row: str(row['project_id']) in row['project_list'],axis=1)
checking_df_merged_filtered = checking_df_merged_filtered[~hasOldProject]

print(checking_df_merged_filtered.shape)soundex
checking_df_merged_filtered.head()

(37653000, 7)
(37594070, 7)
CPU times: user 24min 44s, sys: 3min 48s, total: 28min 32s
Wall time: 28min 18s


In [11]:
checking_df_merged_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37594070 entries, 0 to 37652999
Data columns (total 7 columns):
userCode                        object
project_id                      int64
starting_price_range_user       float64
starting_price_range_project    int64
project_list                    object
most_province_id                int64
province_id_project             int64
dtypes: float64(1), int64(4), object(2)
memory usage: 2.2+ GB


In [12]:
checking_df_merged_filtered.to_csv("./clean_data/deep_features_7000k.csv",index=False)

In [34]:
# Continue here
checking_df_merged_filtered = pd.read_csv("./clean_data/deep_features_7000k.csv")

### Preparing data

In [35]:
# Prepare data for training

#1. Only select userCode and project_id
checking_df_fin = checking_df_merged_filtered[['userCode','project_id']]


#2. Map userCode and project_id to its Idx
checking_df_fet = checking_df_fin.merge(user2IdxDf[['userCode','userIdx']], on = 'userCode',suffixes=("","_user"))
checking_df_fet = checking_df_fet.merge(proj2IdxDf[['project_id','projIdx']], on = 'project_id',suffixes=("_user","_project"))

#3. Concat doc2vec vector by project_id
checking_df_fet3 = checking_df_fet.merge(docvec, on='project_id')


checking_df_fet2 = checking_df_fet3.drop(columns=['userCode','project_id'])

In [14]:
#checking_df_fet3.to_csv("./clean_data/deep_test_7000row_fold0.csv",index=False)

In [ ]:
test = checking_df_fet2.fillna(0)
test_tensors = Variable(torch.Tensor(test.values).float())

In [ ]:
# Test predicting with model
outputs = model(test_tensors).data.numpy()

In [ ]:
checking_df_fet3.head()

## Get top n scores

In [ ]:
%%time
# Rank each user based on their project score

ranking_df = checking_df_fet[['userCode','project_id']]
ranking_df['scores'] = outputs
ranking_df['rank'] = ranking_df.groupby('userCode')['scores'].rank(ascending=False)

In [ ]:
# Get only first seven, then combine them into list (In order!)
selected_ranking_df = ranking_df[ranking_df['rank']<=7]
user_answered_projects = selected_ranking_df.sort_values(["userCode",'rank']).groupby("userCode").agg({"project_id":lambda x:list(x)})

In [ ]:
def_projects = [5858, 3738, 4201, 5452, 5678, 5764, 5067]
def fillProject(project_ids):
    
    if len(project_ids)>=7: return " ".join([str(i) for i in project_ids])
    else: 
        project_ids.extend(def_projects[:7-len(project_ids)])
        return " ".join([str(i) for i in project_ids])
    
fillProject([4743, 6720, 5286, 6727, 8847, 6624])    

user_complete_answered = user_answered_projects
user_complete_answered['project_id'] = user_complete_answered['project_id'].apply(fillProject)

In [ ]:
# Merge testing users with the dataframe, if user doesn't have their ranking, set to default project value.
user_complete_answered.head(10)

In [ ]:
ans = submission_users.merge(user_complete_answered, on='userCode')
ans.head()

In [ ]:
ans.to_csv(submission_file,index=False)